# Créer des tables pivots

Les pivots sont très puissants. Ils vous permettent de synthétiser un tableau de données en choisissant vos lignes, vos colonnes et la manière d’agréger les valeurs.

Il existe deux types de pivots:
- un pivot qui réorganise le tableau, sans modifier les valeurs: `pd.pivot()`
- un pivot qui agrège les valeurs (somme, médiane, moyenne, etc.): `pd.pivot_table()`

In [1]:
import pandas as pd

## 1. Réorganiser avec `pivot()`

Les jeux de données sont souvent organisés sous forme d’enregistrements (données empilées), dans lesquels une colonne correspond à une variable. Par exemple, le fichier Stat-Tab [«Couverture des charges des hautes écoles universitaires»](https://www.bfs.admin.ch/asset/fr/32246244) de l’OFS contient une variable «Source de financement», dont les différentes valeurs sont répétées pour chaque année.

Si on s’intéresse à une haute école en particulier, on peut réorganiser les données pour y voir plus clair.

In [2]:
df = pd.read_csv("data/couverture-charges-hautes-ecoles.csv")

# Filtre avec 2 sources de financement, pour montrer le principe
df[df["Source de financement"].isin(["Ecolage", "Fondations"])].head()

,Année,Source de financement,BS,BE,FR,GE,LS,LU,NE,SG,UZH,USI,EPFL,ETHZ,FS-CH,IHEID
0,1995,Ecolage,7057248,8015360,5144738,3400000,5811377,198405,1921134,3484643,18823722,0,3041000,5591000,...,...
9,1995,Fondations,0,0,0,0,0,0,0,0,0,0,0,0,...,...
38,1996,Ecolage,6729000,12872325,5175000,10094425,6394000,228000,1848133,3503000,20043000,0,3272000,7498000,...,...
47,1996,Fondations,0,0,0,0,0,0,0,0,0,0,0,0,...,...
76,1997,Ecolage,6712489,8187049,6608035,8297000,5728922,258805,1772454,3711403,20123555,0,3185000,7474000,...,...


In [3]:
# Mettons qu’on s’intéresse à l’IHEID. Plusieurs années n’ont pas de données, on va les éliminer
df = df[["Année", "Source de financement", "IHEID"]].copy()
df.head()

,Année,Source de financement,IHEID
0,1995,Ecolage,...
1,1995,Autres moyens propres de la haute école,...
2,1995,Canton universitaire: couverture ou budget,...
3,1995,Canton: imputations internes (jusqu'en 1999),...
4,1995,Autres cantons: accord intercantonal universit...,...


In [4]:
# On élimine les valeurs manquantes...
df = df[df["IHEID"] != "..."].copy()
# ... et les valeurs égales à zéro - que se passe-t-il?
df = df[df["IHEID"] != 0].copy()
df.head(10)

,Année,Source de financement,IHEID
836,2017,Ecolage,5909076
837,2017,Autres moyens propres de la haute école,8546304
838,2017,Canton universitaire: couverture ou budget,15188806
839,2017,Canton: imputations internes (jusqu'en 1999),0
840,2017,Autres cantons: accord intercantonal universit...,765055
841,2017,Autres cantons: autres subventions,0
842,2017,Confédération: contributions aux investissemen...,0
843,2017,Confédération: contributions liées à des proje...,0
844,2017,Confédération: autres subventions,0
845,2017,Fondations,1254536


In [5]:
# A cause des "..." de l’OFS, nos données étaient sous forme de str. On les convertit
df["IHEID"] = df["IHEID"].astype(int)
df = df[df["IHEID"] != 0].copy()
df.head()

,Année,Source de financement,IHEID
836,2017,Ecolage,5909076
837,2017,Autres moyens propres de la haute école,8546304
838,2017,Canton universitaire: couverture ou budget,15188806
840,2017,Autres cantons: accord intercantonal universit...,765055
845,2017,Fondations,1254536


In [6]:
# On peut enfin faire notre pivot
pd.pivot(
    df,
    index="Source de financement",  # Notre nouvel index (ou en-tête de lignes)
    columns="Année",  # Nos nouvelles colonnes
    values="IHEID",  # Nos valeurs. Elles seront déplacées, pas transformées
)

Année,2017,2018,2019
Source de financement,,,
Autres cantons: accord intercantonal universitaire,765055,614280,739350
Autres moyens propres de la haute école,8546304,8299313,7932563
Autres programmes internationaux,1363111,1045514,799482
Canton universitaire: couverture ou budget,15188806,14895798,15036918
Confédération: subventions de base LEHE (autrefois LAU),18500000,18326379,18000000
Ecolage,5909076,6049912,6577521
Fondations,1254536,1441445,1447556
Frais centraux de la haute école couverts par des overheads,3059864,4082757,2270050
Mandats de recherche de la Confédération,7506022,6963630,5881122


## Autre exemple de `pivot()` avec les nuitées dans les hôtels

Le canton est dans la variable "Région touristique"

In [7]:
df = pd.read_csv("data/nuitees-hotellerie.csv", encoding="latin-1")
df.head()

,Année,Mois,Région touristique,Suisse Nuitées,Pays baltes Nuitées,Allemagne Nuitées,France Nuitées,Italie Nuitées,Autriche Nuitées,Royaume-Uni Nuitées,...,Émirats arabes unis Nuitées,Serbie et Monténégro Nuitées,Croatie Nuitées,Roumanie Nuitées,Russie Nuitées,Slovaquie Nuitées,Slovénie Nuitées,Tchéquie Nuitées,Ukraine Nuitées,Serbie Nuitées
0,2005,Total de l'année,Vaud,872962,2214,193683,303084,75671,11177,185955,...,...,2155,1498,5661,29834,984,1177,5825,3527,...
1,2005,Total de l'année,Genève,405296,5702,110755,196975,91335,17404,256363,...,...,3194,3048,9395,54134,2352,2407,4726,9241,...
2,2005,Total de l'année,Valais,1995703,3069,671675,214804,80774,23871,320207,...,...,1917,596,2380,52363,881,1403,5497,2810,...
3,2006,Total de l'année,Vaud,891479,2355,195297,290453,81428,14607,201283,...,...,2422,1931,7257,35740,1726,1675,6941,5500,...
4,2006,Total de l'année,Genève,520467,7673,129677,224247,94867,17577,276059,...,...,3967,3431,11019,60064,2119,2634,6818,10286,...


In [8]:
pd.pivot(
    df,
    index="Année",
    columns="Région touristique",
    values="Croatie Nuitées"
)

Région touristique,Genève,Valais,Vaud
Année,,,
2005,3048,596,1498
2006,3431,496,1931
2007,3397,1758,2152
2008,3722,1031,2055
2009,2678,1005,2909
2010,3130,705,2379
2011,3186,972,2203
2012,2855,1093,1909
2013,3083,599,3430


In [9]:
# Commande à exécuter si la cellule d'après dit:
# "ModuleNotFoundError: No module named 'pyarrow'"
# !pip install pyarrow

In [50]:
# Si cette commande ne marche pas, voir ci-dessus ou me demander de l'aide
df = pd.read_feather("data/ch_autos.feather")
df.head()

,Fahrzeugklasse,Fahrzeugart_Code,Fahrzeugart,Marken_Code,Marke,Marke_und_Typ,Typ1,Typ2,Typ3,Typ4,...,Ersterfassung_Monat,Schildfarbe_Code,Schildfarbe,Schildart_Code,Schildart,Inverkehrsetzung_Status_Code,Inverkehrsetzung_Kanton,PLZ,Staat_Code,Staat
4,M1,1,Personenwagen,3155,ABARTH,Abarth 500 Opening Edition,500 Opening Edition,500,Opening,Edition,...,NaN,0.0,Weiss (Motorwagen),N,Normalschild,I,VD,11..,CH,Schweiz
5,M1,1,Personenwagen,3155,ABARTH,Abarth 500 Opening Edition,500 Opening Edition,500,Opening,Edition,...,NaN,0.0,Weiss (Motorwagen),N,Normalschild,I,VD,12..,CH,Schweiz
12,M1,1,Personenwagen,3155,ABARTH,ABARTH 500e,500e,500e,None,None,...,NaN,0.0,Weiss (Motorwagen),N,Normalschild,I,VS,19..,CH,Schweiz
16,M1,1,Personenwagen,3155,ABARTH,ABARTH 500e,500e,500e,None,None,...,NaN,0.0,Weiss (Motorwagen),N,Normalschild,I,GE,12..,CH,Schweiz
22,M1,1,Personenwagen,3155,ABARTH,ABARTH 500e,500e,500e,None,None,...,NaN,0.0,Weiss (Motorwagen),N,Normalschild,I,VS,39..,CH,Schweiz


In [60]:
# Pour alléger le fichier de données, j'ai conservé uniquement les immatriculations actuellement dans des cantons
# romands et les véhicules militaires
df["Inverkehrsetzung_Kanton"].value_counts()

Inverkehrsetzung_Kanton
VD    603287
VS    350724
GE    316394
FR    291926
NE    135976
JU     69537
M      30418
Name: count, dtype: int64

In [61]:
# Notre fichier contient quand même presque 2 millions de ligne
df.shape

(1798262, 63)

In [69]:
# Il contient tout type de véhicules. On va filtrer
# pour conserver uniquement les voitures
df["Fahrzeugart"].value_counts()

Fahrzeugart
Personenwagen                     1238831
Motorrad                           189678
Lieferwagen                        119693
Sachentransportanhänger             95740
Landwirt. Traktor                   29862
Leichter Motorwagen                 21014
Lastwagen                           15677
Arbeitsanhänger                     10837
Arbeitskarren                        7851
Motorkarren                          5999
Arbeitsmaschine                      5758
Sportgeräteanhänger                  5099
Kleinmotorfahrzeug                   4753
Landw. Motorkarren                   4548
Wohnanhänger                         4520
Landw. Anhänger                      4252
Landw. Arbeitskarren                 3643
Kleinmotorrad                        3598
Sattel-Sachentransportanhänger       3352
Gesellschaftswagen                   2918
Landw. Arbeitsanhänger               2690
Schwerer Motorwagen                  2556
Sattelschlepper                      2398
Anhänger              

In [71]:
df = df[df["Fahrzeugart"] == "Personenwagen"].copy()

In [72]:
# Selection des 10 marques les plus courantes
top10 = df["Marke"].value_counts().head(10).index
top10

Index(['VW', 'AUDI', 'MERCEDES-BENZ', 'BMW', 'TOYOTA', 'SKODA', 'RENAULT',
       'FORD', 'PEUGEOT', 'OPEL'],
      dtype='object', name='Marke')

In [73]:
# Nombre de voitures par marque et par canton
pd.pivot_table(
    df[df["Marke"].isin(top10)],
    index="Marke",
    columns="Inverkehrsetzung_Kanton",
    aggfunc="count",
    values="Fahrzeugklasse",
)

Inverkehrsetzung_Kanton,FR,GE,JU,M,NE,VD,VS
Marke,,,,,,,
AUDI,12037,17948,2631,135,6989,32392,16240
BMW,11821,15388,1990,318,5498,29078,14137
FORD,9756,5087,2823,58,4590,16442,9304
MERCEDES-BENZ,11355,19087,1793,117,4989,28778,17063
OPEL,8605,6334,2291,121,3889,15262,7599
PEUGEOT,7940,7963,2090,985,5099,18310,4852
RENAULT,8880,7448,3068,1480,5145,17897,8971
SKODA,11102,6037,1873,693,4779,18653,11477
TOYOTA,14108,11444,2675,199,4272,24178,11210


In [74]:
print("Nombre moyen de chevaux des voitures immatriculées, par marque, en kW")
pd.pivot_table(
    df[df["Marke"].isin(top10)], index="Marke", values="Leistung", aggfunc="mean"
).round(1)

Nombre moyen de chevaux des voitures immatriculées, par marque, en kW


,Leistung
Marke,
AUDI,156.3
BMW,168.4
FORD,105.5
MERCEDES-BENZ,158.0
OPEL,92.4
PEUGEOT,87.5
RENAULT,88.9
SKODA,109.7
TOYOTA,85.6


In [75]:
# On peut convertir les kW en chevaux en les multipliant par ce nombre:
kw_to_ch = 1.35962162



In [76]:
# Les marques du top 10, par canton
pd.pivot_table(
    df[df["Marke"].isin(top10)],
    index="Marke",
    columns="Inverkehrsetzung_Kanton",
    values="Fahrzeugart",
    aggfunc="count",
).round(1)

Inverkehrsetzung_Kanton,FR,GE,JU,M,NE,VD,VS
Marke,,,,,,,
AUDI,12089,18027,2675,135,7043,32399,16282
BMW,11878,15473,2027,318,5545,29087,14216
FORD,9786,5132,2843,58,4621,16452,9337
MERCEDES-BENZ,11451,19235,1812,117,5019,28807,17166
OPEL,8652,6405,2312,123,3921,15281,7640
PEUGEOT,7976,8019,2128,985,5166,18320,4873
RENAULT,8924,7517,3088,1480,5202,17906,8995
SKODA,11107,6041,1879,693,4797,18653,11483
TOYOTA,14224,11593,2710,199,4316,24198,11335


In [ ]:
# Exemple un peu absurde. Mais on peut voir la puissance
# moyenne par canton et par marque
pd.pivot_table(
    df[df["Marke"].isin(top10)],
    index="Marke",
    columns="Inverkehrsetzung_Kanton",
    values="Leistung",
    aggfunc="mean",
).round(1)

Inverkehrsetzung_Kanton,FR,GE,JU,M,NE,VD,VS
Marke,,,,,,,
AUDI,154.3,150.2,159.5,158.1,154.0,155.0,167.4
BMW,157.0,145.3,156.9,87.6,161.8,155.3,162.0
FORD,104.5,98.9,105.1,105.1,102.6,102.5,108.7
HONDA,66.6,42.2,65.8,130.0,65.3,59.1,64.3
MERCEDES-BENZ,155.1,151.3,160.7,132.7,156.3,154.8,162.4
PEUGEOT,86.7,81.3,84.8,83.6,87.7,84.1,84.1
RENAULT,94.0,88.9,91.4,92.1,89.4,88.1,94.6
SKODA,110.6,106.6,111.8,101.4,106.5,108.2,114.4
TOYOTA,86.4,84.6,88.2,111.7,86.5,84.2,95.1


In [ ]:
# La première mise en circulation par canton
pd.pivot_table(
    df,
    columns="Inverkehrsetzung_Kanton",
    values="Erstinverkehrsetzung_Jahr",
    aggfunc="min",
).round(1)

Inverkehrsetzung_Kanton,FR,GE,JU,M,NE,VD,VS
Erstinverkehrsetzung_Jahr,1912,1900,1908,1942,1908,1900,1903


In [ ]:
# Voitures pre-1920, avec les marques
pd.pivot_table(
    df[df["Erstinverkehrsetzung_Jahr"] < 1920],
    index="Marke",
    columns="Inverkehrsetzung_Kanton",
    values="Erstinverkehrsetzung_Jahr",
    aggfunc="min",
).fillna("")

Inverkehrsetzung_Kanton,FR,GE,JU,NE,VD,VS
Marke,,,,,,
ARIES,,,,1912.0,,
BRASIER,,,,,1913.0,
BUICK,,,1910.0,,,
CHARRON,,1912.0,,,,
CHEVROLET,,1900.0,,,,
CITROEN,,1900.0,,,,
DACIA,,1918.0,,,,
DARRACQ,,,,,1900.0,
DELAGE,1912.0,,,,,
